In [1]:
import os
os.chdir("/home/tangir/crmbm/python/pastis")

from IPython import get_ipython
get_ipython().magic("clear")

import matplotlib.pylab as plt
import matplotlib.dates as mdates
import mrs.reco as reco
import mrs.sim as sim
import mrs.log as log
import mrs.aliases as xxx
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import suspect
from datetime import datetime
import multiprocessing

plt.close("all")
plt.rcParams['figure.dpi'] = 75
plt.rcParams['figure.max_open_warning'] = 1000
plt.rcParams["figure.figsize"] = [8, 6]
# get_ipython().magic("matplotlib inline")
get_ipython().magic("matplotlib notebook")
log.setLevel(log.INFO)

cm = sns.color_palette("rocket", as_cmap=True)
#pd.set_option('display.max_rows', 50)
#pd.options.mode.chained_assignment = None  # default='warn'

(DEBUG) Loaded backend nbAgg version unknown.


# Load dataframes from files

In [2]:
df_sc_all_reco = pd.read_pickle("/home/tangir/crmbm/acq_db/sc_all_reco_notebook.pkl")
df_brain = pd.read_pickle("/home/tangir/crmbm/acq_db/brain_notebook.pkl")

In [ ]:
df_sc_fit = pd.read_pickle("/home/tangir/crmbm/acq_db/sc_fit.pkl")
df_sc_fit["reco_template_name"] = "sc"

df_sc_nodatarej_fit = pd.read_pickle("/home/tangir/crmbm/acq_db/sc_nodatarej_fit.pkl")
df_sc_nodatarej_fit["reco_template_name"] = "sc_nodatarej"

df_sc_nodatarej_norea_fit = pd.read_pickle("/home/tangir/crmbm/acq_db/sc_nodatarej_norea_fit.pkl")
df_sc_nodatarej_norea_fit["reco_template_name"] = "sc_nodatarej_norea"

df_brain_fit = pd.read_pickle("/home/tangir/crmbm/acq_db/brain_fit.pkl")
df_brain_fit["reco_template_name"] = "brain"

# Join dataframes

In [ ]:
df_sc_all_reco = df_sc_all_reco.reset_index()
df_brain = df_brain.reset_index()
#
df_sc_fit = pd.concat([df_sc_fit, df_sc_nodatarej_fit, df_sc_nodatarej_norea_fit])
df_sc_fit = df_sc_fit.reset_index()
#
df_brain_fit = df_brain_fit.reset_index()

In [ ]:
df_sc_all_reco = pd.merge(df_sc_all_reco, df_sc_fit, left_on=["index", "reco_template_name"], right_on=["fit_ws_data_file_hash", "reco_template_name"], how="inner")
df_sc_all_reco = df_sc_all_reco.rename(columns = {'index': 'data_hash'})
df_sc_all_reco = df_sc_all_reco.set_index(["data_hash", "reco_template_name", "fit_ws_fit_hash"])

df_brain = pd.merge(df_brain, df_brain_fit, left_on=["index", "reco_template_name"], right_on=["fit_ws_data_file_hash", "reco_template_name"], how="inner")
df_brain = df_brain.rename(columns = {'index': 'data_hash'})
df_brain = df_brain.set_index(["data_hash", "reco_template_name", "fit_ws_fit_hash"])

# Recalculate metabolite parameters
## T2 correction

In [ ]:
def recalculate_metabolite_concentrations(this_df):
    # fit
    # T2 correction
    this_df["fit_ws_params_fit_t2cor_obj"] = [o.correct_T2s(te) for o,te in zip(this_df["fit_ws_params_fit_obj"], this_df["reco_dataset_raw_data_sequence_te"])]
    this_df["fit_nows_params_fit_t2cor_obj"] = [o.correct_T2s(te) for o,te in zip(this_df["fit_nows_params_fit_obj"], this_df["reco_dataset_raw_data_sequence_te"])]

    return(this_df)

# multiproc run
#pool = multiprocessing.Pool()
#a = pool.map(recalculate_metabolite_concentrations, [df_brain, df_sc_all_reco])
df_brain = recalculate_metabolite_concentrations(df_brain)
df_sc_all_reco = recalculate_metabolite_concentrations(df_sc_all_reco)


## Breakdown params into columns

In [ ]:
cols2breakdown_list = ["fit_ws_params_fit_obj",
                       "fit_ws_params_fit_t2cor_obj",
                       "fit_nows_params_fit_t2cor_obj"]

def breakdown_params_into_columns(this_df):

    # remove all params columns
    this_df = this_df.reset_index()
    cols2remove_list = this_df.columns[this_df.columns.str.contains("\|")].to_list()
    this_df = this_df.drop(cols2remove_list, axis=1)

    # for each column, break down, concat
    this_df_new_columns_list = []
    for c in cols2breakdown_list:
        c_prefix = c[:-3]
        print(c_prefix)

        # break down, keep only concentrations
        this_df_list = [o.to_dataframe(c_prefix).filter(regex="cm\|") if(type(o) is not float) else None for o in this_df[c]]

        this_df_new_columns = pd.concat(this_df_list)

        # add to list of new columns
        this_df_new_columns_list.append(this_df_new_columns.reset_index())

    this_df = pd.concat([this_df] + this_df_new_columns_list, axis=1)
    this_df = this_df.set_index(["data_hash", "reco_template_name", "fit_ws_fit_hash"])
    
    # clean-up: remove index and all params_obj columns
    this_df = this_df.drop("index", axis=1) # from previous reset_index
    this_df = this_df.rename(columns = {"fit_ws_params_fit_obj": "final_fit_params_arr"})
    cols2remove = this_df.columns[this_df.columns.str.contains("_params_") & this_df.columns.str.endswith("_obj")]
    this_df = this_df.drop(cols2remove, axis=1)

    return(this_df)

df_brain = breakdown_params_into_columns(df_brain)
df_sc_all_reco = breakdown_params_into_columns(df_sc_all_reco)


## Combine some metabolite estimates

In [ ]:
combined_metabolites_dict = {"tNAA": ["NAA", "NAAG"],
                             "tCr": ["Cr_CH3", "PCr"],
                             "tCho": ["GPC", "PC"]}  

def additional_columns(this_df):

    for this_param_type in cols2breakdown_list:
        print(this_param_type)
        for this_meta_parent, this_meta_children_list in combined_metabolites_dict.items():

            # init concentration
            this_meta_parent_key = this_param_type[:-3] + "cm|" + this_meta_parent + "|" + "val"
            this_df[this_meta_parent_key] = 0.0
            print(this_meta_parent_key)

            # summing up estimated concentrations
            for this_meta_child in this_meta_children_list:
                this_meta_child_key = this_param_type[:-3] + "cm|" + this_meta_child + "|" + "val"
                this_df[this_meta_parent_key] += this_df[this_meta_child_key]

            # init error
            this_meta_parent_key = this_param_type[:-3] + "cm|" + this_meta_parent + "|" + "err"
            this_df[this_meta_parent_key] = 0.0
            print(this_meta_parent_key)

            # summing up squared of errors
            for this_meta_child in this_meta_children_list:
                this_meta_child_key = this_param_type[:-3] + "cm|" + this_meta_child + "|" + "err"
                this_df[this_meta_parent_key] += this_df[this_meta_child_key]**2

            # sqrt the sum of squares
            this_df[this_meta_parent_key] = [np.sqrt(e) for e in this_df[this_meta_parent_key]]

    return(this_df)

df_brain = additional_columns(df_brain)
df_sc_all_reco = additional_columns(df_sc_all_reco)

## Calculate metabolite absolute and ratio concentrations

In [ ]:
water_concentration = 55000.0

def recalculate_metabolite_abs_ratios(this_df):

    fit_ws_params_fit_t2cor_columns_list = this_df.columns[this_df.columns.str.startswith("fit_ws_params_fit_t2cor_cm") & this_df.columns.str.endswith("|val")].tolist()

    for this_col in fit_ws_params_fit_t2cor_columns_list:
        
        # calculate relative CRB errors in (%)
        this_err_col = this_col.replace("|val", "|err")
        this_err_prct_col = this_col.replace("|val", "|err_prct")
        this_df[this_err_prct_col] = this_df[this_err_col] / this_df[this_col] * 100.0

        # absolute concentrations rel. to water
        this_abs_val_col = this_col.replace("fit_ws_params_fit_t2cor_cm", "fit_ws_params_fit_t2cor_abs_cm")
        this_water_val_col = "fit_nows_params_fit_t2cor_cm|Water|val"
        this_df[this_abs_val_col] = this_df[this_col] * water_concentration / this_df[this_water_val_col]
        # and its error: propagating metabolite/water errors
        this_err_col = this_col.replace("|val", "|err")
        this_abs_err_col = this_col.replace("fit_ws_params_fit_t2cor_cm", "fit_ws_params_fit_t2cor_abs_cm").replace("|val", "|err")
        this_abs_err_prct_col = this_col.replace("fit_ws_params_fit_t2cor_cm", "fit_ws_params_fit_t2cor_abs_cm").replace("|val", "|err_prct")
        this_water_err_col = "fit_nows_params_fit_t2cor_cm|Water|err"
        this_df[this_abs_err_col] = [met_abs_cm * np.sqrt((met_err/met_cm)**2 + (water_err/water_cm)**2) if((met_cm > 0) & (water_cm>0)) else np.nan for (met_abs_cm, met_cm, met_err, water_cm, water_err) in zip(this_df[this_abs_val_col], this_df[this_col], this_df[this_err_col], this_df[this_water_val_col], this_df[this_water_err_col])]   
        this_df[this_abs_err_prct_col] = this_df[this_abs_err_col] / this_df[this_abs_val_col] * 100.0
        
        # ratio concentrations rel. to tCr
        this_ratio_val_col = this_col.replace("fit_ws_params_fit_t2cor_cm", "fit_ws_params_fit_t2cor_ratio_cm")
        this_tcr_val_col = "fit_ws_params_fit_t2cor_cm|tCr|val"
        this_df[this_ratio_val_col] = this_df[this_col] / this_df[this_tcr_val_col]
        # and its error: propagating metabolite/tCr errors
        this_err_col = this_col.replace("|val", "|err")
        this_ratio_err_col = this_col.replace("fit_ws_params_fit_t2cor_cm", "fit_ws_params_fit_t2cor_ratio_cm").replace("|val", "|err")
        this_ratio_err_prct_col = this_col.replace("fit_ws_params_fit_t2cor_cm", "fit_ws_params_fit_t2cor_ratio_cm").replace("|val", "|err_prct")
        this_tcr_err_col = "fit_ws_params_fit_t2cor_cm|tCr|err"
        this_df[this_ratio_err_col] = [met_ratio_cm * np.sqrt((met_err/met_cm)**2 + (tcr_err/tcr_cm)**2) if((met_cm > 0) & (tcr_cm>0)) else np.nan for (met_ratio_cm, met_cm, met_err, tcr_cm, tcr_err) in zip(this_df[this_ratio_val_col], this_df[this_col], this_df[this_err_col], this_df[this_tcr_val_col], this_df[this_tcr_err_col])]   
        this_df[this_ratio_err_prct_col] = this_df[this_ratio_err_col] / this_df[this_ratio_val_col] * 100.0

    return(this_df)

df_brain = recalculate_metabolite_abs_ratios(df_brain)
df_sc_all_reco = recalculate_metabolite_abs_ratios(df_sc_all_reco)

# Correlation matrix analysis
In case I missed something interesting!

In [ ]:
def list_highest_corr(this_df_corr, column_name):

    this_df_corr2 = this_df_corr[column_name]
    this_df_corr2 = this_df_corr2.loc[this_df_corr2.abs() < (1.00 - 1e-6)]
    this_df_corr2 = this_df_corr2.loc[this_df_corr2.abs() > 0.5]
    this_df_corr2 = this_df_corr2.iloc[(-this_df_corr2.abs()).argsort()]
    return(this_df_corr2)

df = df_sc_all_reco
# correlate only numeric fields
df_num = df.select_dtypes(include=np.number)   
df_corr = df_num.corr(min_periods=5)
pd.set_option('display.max_rows', 500)


In [ ]:
this_df_corr = list_highest_corr(df_corr, "snr_final")
this_df_corr

* The SNR is correlated with:
    * water signal intensity
    * phase, amplitude std (negative)

In [ ]:
this_df_corr = list_highest_corr(df_corr, "patient_bmi")
this_df_corr

* BMI is correlated with:
    * patient weight/height etc.
    * f0 frequency !
    * voxel Y position
    * sequence TR
    * CSDEs
    * SNR (negative)
    * sequence Vref
    * sequence pulse duration
    * shim n*4 (ZX)

In [ ]:
this_df_corr = list_highest_corr(df_corr, "lw_final")
this_df_corr

* Linewidth is correlated with:
    * some small metabolites error
    * metabolites damping factors
    * prejection linewidth std

In [ ]:
this_df_corr = list_highest_corr(df_corr, "reco_dataset_raw_data_sequence_vref")
this_df_corr

* Vref is correlated with:
    * sequence TR
    * sequence pulse duration
    * CSDEs
    * voxel Y position (negative)
    * patient BMI/weight/height etc.
    * acquisition duration
    * shims (sum) (negative)
    * shims (2nd order) (negative)

In [ ]:
this_df_corr = list_highest_corr(df_corr, "reco_dataset_raw_data_sequence_te")
this_df_corr

* TE is correlated with:
    * sequence pulse duration
    * sequence pulse R
    * sequence pulse N (negative)

In [ ]:
this_df_corr = list_highest_corr(df_corr, "fit_ws_params_fit_cm|NAA_CH3|err")
this_df_corr

* CRB(NAA) is correlated with:
    * other metabolites' CRBs
    * no acquisition parameters

In [ ]:
pd.set_option('display.max_rows', 50)

# Free some memory

In [ ]:
except_list = ["df_sc_all_reco", "df_brain"]

for var_name in dir():
    if(eval("type(" + var_name + ")") == pd.DataFrame and var_name not in except_list):
        del globals()[var_name]


# Additional columns

In [ ]:
def additional_columns(this_df):
    this_df["fit_ws_metabolites_len"] = [len(mbl) if (type(mbl) is not float) else 1 for mbl in this_df["fit_ws_metabolites"]]
    this_df["fit_ws_sequence_str"] = ["press" if("press" in s) else "none" for s in this_df["fit_ws_sequence"].astype(str)]
    this_df["fit_tool_name"] = ["pastis" if("pastis" in s) else "lcmodel" for s in this_df["fit_ws_name"]]
    
    this_df["strategy"] = this_df.index.get_level_values("reco_template_name").astype(str) + "|" + \
                this_df["fit_tool_name"].astype(str) + "|" + \
                this_df["fit_ws_metabolites_len"].astype(str) + "|" + \
                this_df["fit_ws_sequence_str"].astype(str)
    
    return(this_df)

df_brain = additional_columns(df_brain)
df_sc_all_reco = additional_columns(df_sc_all_reco)

# Convert to a long dataframe
in order to be able to do seaborn plots easily

In [ ]:
def convert_to_long(this_df):
    
    this_df = this_df.reset_index()
    
    # find all columns not related to params
    id_vars_list = this_df.columns[~this_df.columns.str.contains("\|")]

    # melt all those param columns
    this_df = this_df.melt(id_vars=id_vars_list, var_name="melted_params_name", value_name="param_val")         

    # split columns into metabolite, parameter name, err, etc.
    melted_params_name_col = this_df["melted_params_name"]
    this_df["param_p_type"] = melted_params_name_col.str.split("|").str[0]
    this_df["param_m_name"] = melted_params_name_col.str.split("|").str[1]
    this_df["param_val_err"] = melted_params_name_col.str.split("|").str[2]
    this_df = this_df.drop("melted_params_name", axis=1)
    
    this_df = this_df.set_index(["data_hash", "reco_template_name", "fit_ws_fit_hash"])

    return(this_df)

df_brain = convert_to_long(df_brain)
df_sc_all_reco = convert_to_long(df_sc_all_reco)


# Optimal fit strategy

* Which sequence for simulation?
* How many metabolites?

## Brain data

### Fit strategies vs R2/FQN

In [ ]:
df = df_brain.loc[(df_brain.index.get_level_values("reco_template_name") == "brain") &
                  (df_brain["fit_tool_name"] == "pastis")]

plt.figure()
g = sns.boxplot(x="fit_ws_metabolites_len",  
                y="fit_ws_optim_results_fqn",
                hue="fit_ws_sequence_str",
                data=df)

In [ ]:
plt.figure()
g = sns.boxplot(x="fit_ws_metabolites_len", 
                y="fit_ws_optim_results_rsq_f",
                hue="fit_ws_sequence_str",
                data=df)

plt.figure()
g = sns.boxplot(x="fit_ws_metabolites_len", 
                y="fit_ws_optim_results_rsq_t",
                hue="fit_ws_sequence_str",
                data=df)


* Evidemment, the more we include metabolites in the fit basis set, the better is the fit in terms of FQN, R2, etc...
* sLASER ("none") should be use for the signal simulation/fit

### Fit strategies vs CRBs

In [ ]:
df = df_brain.loc[(df_brain.index.get_level_values("reco_template_name") == "brain")]

# watch CRBs
this_df = df.loc[(df["fit_tool_name"] == "pastis") &
                 (df["param_p_type"] == "fit_ws_params_fit_t2cor_cm") &
                 (df["param_val_err"] == "err_prct") &  
                 (df["param_m_name"].isin(["tCr", "tCho", "tNAA", "Glu", "Gln", "mI"]))]

g = sns.catplot(x="param_m_name", 
                y="param_val",
                hue="fit_ws_sequence_str", col="fit_ws_metabolites_len", kind="box",
                data=this_df)

plt.ylim([0, 50])

sLASER or PRESS gives really similar CRBs...

In [ ]:
# keep only pastis/sLASER and lcmodel
df = df_brain.loc[(df_brain.index.get_level_values("reco_template_name") == "brain") &
                        (df_brain["fit_ws_sequence_str"] == "none")]

# watch CRBs
this_df = df.loc[(df["param_p_type"] == "fit_ws_params_fit_t2cor_cm") &
                 (df["param_val_err"] == "err_prct") &  
                 (df["param_m_name"].isin(["tCr", "tCho", "tNAA", "Glu", "Gln", "mI"]))]

g = sns.catplot(x="param_m_name", 
                y="param_val",
                hue="fit_ws_metabolites_len", col="fit_tool_name", kind="box",
                data=this_df)

plt.ylim([0, 50])

* LCModel gives higher CRBs in average than PASTIS!
* The number of metabolites is not really affecting the CRBs of the main metabolites (in the brain here) 

### Fit strategies vs Metabolite ratios

In [ ]:
# keep only pastis/sLASER and lcmodel
df = df_brain.loc[(df_brain.index.get_level_values("reco_template_name") == "brain") &
                        (df_brain["fit_ws_sequence_str"] == "none")]

# watch CRBs
this_df = df.loc[(df["param_p_type"] == "fit_ws_params_fit_t2cor_ratio_cm") &
                 (df["param_val_err"] == "val") &  
                 (df["param_m_name"].isin(["tCr", "tCho", "tNAA", "Glu", "Gln", "mI"]))]

g = sns.catplot(x="param_m_name", 
                y="param_val",
                hue="fit_ws_metabolites_len", col="fit_tool_name", kind="box",
                data=this_df)


* On brain data, LCModel shows higher variability for some metabolite ratios :)
* Regarding number of metabolites to fit, hard to conclude. Better look at the CRBs?

## Spinal cord data

### Fit strategies vs R2/FQN

In [ ]:
df = df_sc_all_reco.loc[(df_sc_all_reco.index.get_level_values("reco_template_name") == "sc") &
                        (df_sc_all_reco["fit_tool_name"] == "pastis")]

plt.figure()
g = sns.boxplot(x="fit_ws_metabolites_len",  
                y="fit_ws_optim_results_fqn",
                hue="fit_ws_sequence_str",
                data=df)

In [ ]:
plt.figure()
g = sns.boxplot(x="fit_ws_metabolites_len", 
                y="fit_ws_optim_results_rsq_f",
                hue="fit_ws_sequence_str",
                data=df)

plt.figure()
g = sns.boxplot(x="fit_ws_metabolites_len", 
                y="fit_ws_optim_results_rsq_t",
                hue="fit_ws_sequence_str",
                data=df)

* Evidemment, the more we include metabolites in the fit basis set, the better is the fit in terms of FQN, R2, etc...
* sLASER ("none") should be use for the signal simulation/fit

### Fit strategies vs CRBs

In [ ]:
# keep only pastis/sLASER and lcmodel
df = df_sc_all_reco.loc[(df_sc_all_reco.index.get_level_values("reco_template_name") == "sc") &
                        (df_sc_all_reco["fit_ws_sequence_str"] == "none")]

# watch CRBs
this_df = df.loc[(df["param_p_type"] == "fit_ws_params_fit_t2cor_cm") &
                 (df["param_val_err"] == "err_prct") &  
                 (df["param_m_name"].isin(["tCr", "tCho", "tNAA", "Glu", "Gln", "mI"]))]

g = sns.catplot(x="param_m_name", 
                y="param_val",
                hue="fit_ws_metabolites_len", col="fit_tool_name", kind="box",
                data=this_df)

plt.ylim([0, 100])

* Yes, just like on brain data, LCModel gives higher CRBs than PASTIS!
* Seems that n=21 metabolites could be a bit too much and is affecting the CRBs of tCho and Gln?

### Fit strategies vs Metabolite ratios

In [ ]:
# keep only pastis/sLASER and lcmodel
df = df_sc_all_reco.loc[(df_sc_all_reco.index.get_level_values("reco_template_name") == "sc") &
                        (df_sc_all_reco["fit_ws_sequence_str"] == "none")]

# watch CRBs
this_df = df.loc[(df["param_p_type"] == "fit_ws_params_fit_t2cor_ratio_cm") &
                 (df["param_val_err"] == "val") &  
                 (df["param_m_name"].isin(["tCr", "tCho", "tNAA", "Glu", "Gln", "mI"]))]

g = sns.catplot(x="param_m_name", 
                y="param_val",
                hue="fit_ws_metabolites_len", col="fit_tool_name", kind="box",
                data=this_df)

plt.ylim([0, 4])

* We have clearly different results with PASTIS and LCM... 
* Variability is stronger for some metabolites with LCM

# Data quality (SNR & LW)
## Annotated SNR vs. LW

In [ ]:
def add_labels(key_x, key_y, key_label, this_df_labels, ax, font_size=10, x_offset=0.0, y_offset=0.0):
    for i, pt in this_df_labels.iterrows():
        ax.text(pt[key_x] + .02 + x_offset, pt[key_y] + y_offset, str(pt[key_label]), fontsize=font_size)

In [ ]:
threshold_snr_test = 5
threshold_lw_test = 20

df = df_sc_all_reco.loc[(df_sc_all_reco.index.get_level_values("reco_template_name") == "sc") &
                        (df_sc_all_reco["fit_tool_name"] == "pastis") &
                        (df_sc_all_reco["fit_ws_metabolites_len"] == 17) &
                        (df_sc_all_reco["fit_ws_sequence_str"] == "none")]

this_df = df.drop_duplicates("patient_pass_id")

plt.figure()
g = sns.scatterplot(x="lw_final", y="snr_final", data=this_df)
add_labels("lw_final", "snr_final", "patient_pass_id", this_df, plt.gca())
plt.axvline(threshold_lw_test)
plt.axhline(threshold_snr_test)
plt.grid('on')

## Annotated R2 & FQN

In [ ]:
threshold_fqn_test = 2.2

df = df_sc_all_reco.loc[(df_sc_all_reco.index.get_level_values("reco_template_name") == "sc") &
                        (df_sc_all_reco["fit_tool_name"] == "pastis") &
                        (df_sc_all_reco["fit_ws_metabolites_len"] == 17) &
                        (df_sc_all_reco["fit_ws_sequence_str"] == "none")]

this_df = df.drop_duplicates("patient_pass_id")

plt.figure()
g = sns.scatterplot(x="fit_ws_optim_results_rsq_f", y="fit_ws_optim_results_fqn", data=this_df)
add_labels("fit_ws_optim_results_rsq_f", "fit_ws_optim_results_fqn", "patient_pass_id", this_df, plt.gca())
plt.axhline(threshold_fqn_test)
plt.grid('on')


## CRBs vs. SNR & LW

In [ ]:
df = df_sc_all_reco.loc[(df_sc_all_reco.index.get_level_values("reco_template_name") == "sc") &
                        (df_sc_all_reco["fit_tool_name"] == "pastis") &
                        (df_sc_all_reco["fit_ws_metabolites_len"] == 17) &
                        (df_sc_all_reco["fit_ws_sequence_str"] == "none")]

# watch CRBs
this_df = df.loc[(df["param_p_type"] == "fit_ws_params_fit_t2cor_cm") &
                 (df["param_val_err"] == "err_prct") &  
                 (df["param_m_name"].isin(["tCr", "tCho", "tNAA", "mI"]))]

# sort by SNR
this_df = this_df.sort_values("snr_final")

plt.figure()
g = sns.lineplot(x="snr_final", y="param_val", hue="param_m_name", data=this_df)

plt.ylim([0, 100])

## Metabolite ratios vs. SNR & LW

In [ ]:
df = df_sc_all_reco.loc[(df_sc_all_reco.index.get_level_values("reco_template_name") == "sc") &
                        (df_sc_all_reco["fit_tool_name"] == "pastis") &
                        (df_sc_all_reco["fit_ws_metabolites_len"] == 17) &
                        (df_sc_all_reco["fit_ws_sequence_str"] == "none")]

# watch ratios
this_df = df.loc[(df["param_p_type"] == "fit_ws_params_fit_t2cor_ratio_cm") &
                 (df["param_val_err"] == "val") &  
                 (df["param_m_name"].isin(["tCr", "tCho", "tNAA", "mI"]))]

# sort by SNR
this_df = this_df.sort_values("snr_final")

plt.figure()
g = sns.lineplot(x="snr_final", y="param_val", hue="param_m_name", data=this_df)

plt.ylim([0, 5])

Well yeah, we do have crazy metabolite concentration ratios when SNR is low but it gets better for higher SNR. Makes sense.

# Data quality filtering by SNR & LW & FQN
and take care of orphan patients, etc.

In [ ]:
threshold_snr_final = 5
threshold_lw_final = 20
threshold_fqn_final = 200

ref_reco_template_name_sc = "sc"
ref_reco_template_name_brain = "brain"
ref_fit_tool_name = "pastis"
ref_fit_ws_metabolites_len = 17
ref_fit_ws_sequence_str = "none"

def filter_by_SNR_LW_FQN(this_df, 
                         this_threshold_snr_final, 
                         this_threshold_lw_final, 
                         this_threshold_fqn_final,
                         this_ref_reco_template_name, 
                         this_ref_fit_tool_name, 
                         this_ref_fit_ws_metabolites_len, 
                         this_ref_fit_ws_sequence_str):

    # add a column
    this_df["included"] = False
    this_df["pass_after_filter"] = this_df["pass"]
    this_df["patient_pass_id_after_filter"] = this_df["patient_pass_id"]
    this_df["patient_pass_id_pretty_after_filter"] = this_df["patient_pass_id_pretty"]

    # keep only the reference reco_template, fit stategies, etc.
    this_df_ref = this_df.loc[(this_df.index.get_level_values("reco_template_name") == this_ref_reco_template_name) &
                            (this_df["fit_tool_name"] == this_ref_fit_tool_name) &
                            (this_df["fit_ws_metabolites_len"] == this_ref_fit_ws_metabolites_len) &
                            (this_df["fit_ws_sequence_str"] == this_ref_fit_ws_sequence_str)]

    # apply SNR/LW/FQN filter
    this_df_ref_filtered = this_df_ref.loc[(this_df_ref["snr_final"] > this_threshold_snr_final) &
                                   (this_df_ref["lw_final"] < this_threshold_lw_final) &
                                   (this_df_ref["fit_ws_optim_results_fqn"] < this_threshold_fqn_final)]

    # remember the data hash indexes to keep
    data_hash_2keep_list = this_df_ref_filtered.index.get_level_values("data_hash").unique().tolist()

    # find pass 1 patients
    this_df_ref_filtered_p1 = this_df_ref_filtered.loc[(this_df_ref_filtered["pass"] == 1)]
    # remember them
    patient_id_p1_list = this_df_ref_filtered_p1["patient_id"].unique().tolist()

    # find pass 2 patients which are not previously found pass 1 list
    this_df_ref_filtered_p2_orphan = this_df_ref_filtered.loc[ (this_df_ref_filtered["pass"] == 2) &
                            ~this_df_ref_filtered["patient_id"].isin(patient_id_p1_list) ]

    # remember their ids
    orphan_patient_id_list = list(this_df_ref_filtered_p2_orphan["patient_id"].unique())
    print("found %d orphan P2 patients. Fixing them..." % len(orphan_patient_id_list))
    print(orphan_patient_id_list)

    # apply filter to whole df
    this_df.loc[this_df.index.get_level_values("data_hash").isin(data_hash_2keep_list), "included"] = True

    # transform the orphan pass 2 patients into pass 1 and rename them
    this_df.loc[this_df["patient_id"].isin(orphan_patient_id_list), "pass_after_filter"] = 1
    this_df.loc[this_df["patient_id"].isin(orphan_patient_id_list), "patient_pass_id_after_filter"] = this_df.loc[this_df["patient_id"].isin(orphan_patient_id_list)]["patient_pass_id"].str.replace("P2", "P1")
    this_df.loc[this_df["patient_id"].isin(orphan_patient_id_list), "patient_pass_id_pretty_after_filter"] = this_df.loc[this_df["patient_id"].isin(orphan_patient_id_list)]["patient_pass_id_pretty"].str.replace("P2", "P1")

    # print stuff
    nP1_orig = len(this_df.loc[(this_df["pass"] == 1)].drop_duplicates("patient_id"))
    nP2_orig = len(this_df.loc[(this_df["pass"] == 2)].drop_duplicates("patient_id"))
    nP1_after_filter = len(this_df.loc[this_df["included"] & (this_df["pass_after_filter"] == 1)].drop_duplicates("patient_id"))
    nP2_after_filter = len(this_df.loc[this_df["included"] & (this_df["pass_after_filter"] == 2)].drop_duplicates("patient_id"))
    prct_reject = nP1_after_filter / nP1_orig * 100
    print("n (P1) = %d / %d (%.2f%% included)" % (nP1_after_filter, nP1_orig, prct_reject))
    print("n (P2) = %d / %d" % (nP2_after_filter, nP2_orig))

    return(this_df)

df_sc_all_reco = filter_by_SNR_LW_FQN(df_sc_all_reco, 
                                        threshold_snr_final,
                                        threshold_lw_final,
                                        threshold_fqn_final,
                                        ref_reco_template_name_sc, 
                                        ref_fit_tool_name, 
                                        ref_fit_ws_metabolites_len, 
                                        ref_fit_ws_sequence_str)

df_brain = filter_by_SNR_LW_FQN(df_brain, 
                                        threshold_snr_final,
                                        threshold_lw_final,
                                        threshold_fqn_final,
                                        ref_reco_template_name_brain, 
                                        ref_fit_tool_name, 
                                        ref_fit_ws_metabolites_len, 
                                        ref_fit_ws_sequence_str)


# [FIG] SNR & LW & FQN scatter plot

In [ ]:
df = df_sc_all_reco.loc[(df_sc_all_reco.index.get_level_values("reco_template_name") == "sc") &
                        (df_sc_all_reco["fit_tool_name"] == "pastis") &
                        (df_sc_all_reco["fit_ws_metabolites_len"] == 17) &
                        (df_sc_all_reco["fit_ws_sequence_str"] == "none")].drop_duplicates("patient_pass_id")

print("There is %d datasets included" % len(df.loc[df["included"]]))
print("There is %d datasets excluded" % len(df.loc[~df["included"]]))
print("Datasets rejected (%%) = %.2f" % (len(df.loc[~df["included"]]) / len(df) * 100))

print("--- Included datasets ---")
print("Mean SNR = %.2f (%.2f)" % (df.loc[df["included"]]["snr_final"].mean(),
                                    df.loc[df["included"]]["snr_final"].std()))
print("Mean FWHM = %.2f Hz (%.2f)" % (df.loc[df["included"]]["lw_final"].mean(),
                                        df.loc[df["included"]]["lw_final"].std()))
print("Mean FQN = %.2f (%.2f)" % (df.loc[df["included"]]["fit_ws_optim_results_fqn"].mean(), 
                                    df.loc[df["included"]]["fit_ws_optim_results_fqn"].std()))


In [ ]:
this_df = df

# making an empty col for Included/Excluded
this_df[" "] = "Included"
this_df.loc[~this_df["included"], " "] = "Excluded"

# add best brain dataset
this_df_brain = df_brain.loc[(df_brain.index.get_level_values("reco_template_name") == "brain") &
                        (df_brain["fit_tool_name"] == "pastis") &
                        (df_brain["fit_ws_metabolites_len"] == 17) &
                        (df_brain["fit_ws_sequence_str"] == "none")].drop_duplicates("patient_pass_id")
this_df_brain = this_df_brain.loc[ (this_df_brain["snr_final"] == this_df_brain["snr_final"].max())]

this_df_brain = this_df_brain.drop_duplicates("patient_pass_id")
this_df_brain[" "] = "Included"

# concat and rename scale labels
this_df = pd.concat([this_df, this_df_brain])
this_df = this_df.rename(columns = {'snr_final':'SNR (u.a)'})
this_df = this_df.rename(columns = {'lw_final':'FWHM (Hz)'})
this_df = this_df.rename(columns = {'fit_ws_optim_results_fqn':'FQN (u.a)'})

print("n = %d" % len(this_df))

# broken axis scatter plot
#cm = sns.color_palette("rocket_r", as_cmap=True)
f, (ax_top, ax_bottom) = plt.subplots(ncols=1, nrows=2, sharex=True,
                                      gridspec_kw={'hspace':0.10, 'height_ratios':[1, 10]})
g = sns.scatterplot(x="FWHM (Hz)", y="SNR (u.a)",
                color="black", #hue="FQN (u.a)"
                style=" ", style_order=["Included", "Excluded"],
                s=150, data=this_df, ax=ax_top, legend=False)

ax_top.grid(True)
ax_top.axhline(threshold_snr_final, color='r', linestyle='--')
ax_top.axvline(threshold_lw_final, color='r', linestyle='--')

this_df_labels = this_df.loc[(this_df["SNR (u.a)"] > 50)]
this_df_labels = this_df_labels.drop_duplicates("patient_pass_id")
add_labels("FWHM (Hz)", "SNR (u.a)", "patient_pass_id_pretty", this_df_labels, ax_top, 8, 0.5, 0.0)

g = sns.scatterplot(x="FWHM (Hz)", y="SNR (u.a)",
                color="black", #hue="FQN (u.a)"
                style=" ", style_order=["Included", "Excluded"],
                s=150, data=this_df, ax=ax_bottom)
ax_bottom.grid(True)
ax_bottom.axhline(threshold_snr_final, color='r', linestyle='--')
ax_bottom.axvline(threshold_lw_final, color='r', linestyle='--')

this_df_labels = this_df.loc[(this_df["SNR (u.a)"] < 50)]
this_df_labels = this_df_labels.drop_duplicates("patient_pass_id")
add_labels("FWHM (Hz)", "SNR (u.a)", "patient_pass_id_pretty", this_df_labels, ax_bottom, 8, 0.5, 0.0)

ax_top.set_ylim([90, 110])
ax_top.set_ylabel("")
ax_bottom.set_ylim(4, 20)
sns.despine(ax=ax_bottom)
sns.despine(ax=ax_top, bottom=True)

# draw the small break sign
ax = ax_top
d = .01
kwargs = dict(transform=ax.transAxes, color='k', clip_on=False)
ax.plot((-d, +d), (-d*10, +d*10), **kwargs)
ax2 = ax_bottom
kwargs.update(transform=ax2.transAxes)
ax2.plot((-d, +d), (1 - d, 1 + d), **kwargs)

plt.savefig("./notebooks/figs/fig_snr_lw.svg")

this_df[["patient_pass_id", "included"]]

# Stats: data acquisition, quality
## Height & Weight & BMI

In [ ]:
print("--- before filtering ---")
df = df_sc_all_reco.loc[(df_sc_all_reco["pass"] == 1)].drop_duplicates("patient_id")

print("n = %d" % len(df))

print("Mean weight = %.2fkgs +/-%.2f" % (df["patient_weight"].mean(), df["patient_weight"].std()))
print("Mean height = %.2fm +/-%.2f" % (df["patient_height"].mean(), df["patient_height"].std()))
print("Mean BMI = %.2f +/-%.2f" % (df["patient_bmi"].mean(), df["patient_bmi"].std()))

sex_counts = df.groupby("patient_sex")
print("Sex = M(%d) F(%d)" % (df.loc[sex_counts.groups["M"]]["patient_id"].nunique(), 
                             df.loc[sex_counts.groups["F"]]["patient_id"].nunique()))

In [ ]:
print("--- after filtering ---")
df = df_sc_all_reco.loc[(df_sc_all_reco["pass"] == 1) &
                        (df_sc_all_reco["included"] == True)].drop_duplicates("patient_id")

print("n = %d" % len(df))

print("Mean weight = %.2fkgs +/-%.2f" % (df["patient_weight"].mean(), df["patient_weight"].std()))
print("Mean height = %.2fm +/-%.2f" % (df["patient_height"].mean(), df["patient_height"].std()))
print("Mean BMI = %.2f +/-%.2f" % (df["patient_bmi"].mean(), df["patient_bmi"].std()))

sex_counts = df.groupby("patient_sex")
print("Sex = M(%d) F(%d)" % (df.loc[sex_counts.groups["M"]]["patient_id"].nunique(), 
                             df.loc[sex_counts.groups["F"]]["patient_id"].nunique()))

## Average voxel size

In [ ]:
df = df_sc_all_reco.drop_duplicates(["patient_pass_id"])

print("n = %d" % len(df))

print("* Voxel volume (cm3) = %.2f (%.2f)" % (df["voxel_vol_cm3"].mean(), 
                                              df["voxel_vol_cm3"].std()))

print("* Voxel X dimension (mm) = %.2f (%.2f)" % (df["voxel_vol_cm3"].mean(), 
                                              df["voxel_vol_cm3"].std()))
print("* Voxel Y dimension (mm) = %.2f (%.2f)" % (df["voxel_vol_cm3"].mean(), 
                                              df["voxel_vol_cm3"].std()))
print("* Voxel Z dimension (mm) = %.2f (%.2f)" % (df["voxel_vol_cm3"].mean(), 
                                              df["voxel_vol_cm3"].std()))

In [ ]:
df["voxel_size_x_mm"]

In [ ]:
df["voxel_size_y_mm"]

In [ ]:
df["voxel_size_z_mm"]

## CSDE
Chemical shift displacement errors?

In [ ]:
print("n = %d" % len(df))

print("* X dimension (%%/ppm) = %.2f" % df["csde_x"].mean())
print("* Y dimension (%%/ppm) = %.2f" % df["csde_y"].mean())
print("* Z dimension (%%/ppm) = %.2f" % df["csde_z"].mean())

print("* For protocol(i)?")
print("* X dimension (%%/ppm) = %.2f" % df.loc[df["reco_dataset_raw_data_sequence_pulse_rfc_r"] >=20]["csde_x"].mean())
print("* Y dimension (%%/ppm) = %.2f" % df.loc[df["reco_dataset_raw_data_sequence_pulse_rfc_r"] >=20]["csde_y"].mean())
print("* Z dimension (%%/ppm) = %.2f" % df.loc[df["reco_dataset_raw_data_sequence_pulse_rfc_r"] >=20]["csde_z"].mean())

print("* For protocol(ii)?")
print("* X dimension (%%/ppm) = %.2f" % df.loc[df["reco_dataset_raw_data_sequence_pulse_rfc_r"] <20]["csde_x"].mean())
print("* Y dimension (%%/ppm) = %.2f" % df.loc[df["reco_dataset_raw_data_sequence_pulse_rfc_r"] <20]["csde_y"].mean())
print("* Z dimension (%%/ppm) = %.2f" % df.loc[df["reco_dataset_raw_data_sequence_pulse_rfc_r"] <20]["csde_z"].mean())

df["csde_x"]

In [ ]:
df["csde_y"]

In [ ]:
df["csde_z"]

## Average TE

In [ ]:
print("n = %d" % len(df))

print("* TE (ms) = %.2f (%.2f) [%.2f - %.2f]" % (df["reco_dataset_raw_data_sequence_te"].mean(), 
                                                 df["reco_dataset_raw_data_sequence_te"].std(), 
                                                 df["reco_dataset_raw_data_sequence_te"].min(), 
                                                 df["reco_dataset_raw_data_sequence_te"].max()))

print("* For protocol(i)?")
print("* TE (ms) = %.2f (%.2f) [%.2f - %.2f]" % (df.loc[df["reco_dataset_raw_data_sequence_pulse_rfc_r"] >=20]["reco_dataset_raw_data_sequence_te"].mean(),
                                                 df.loc[df["reco_dataset_raw_data_sequence_pulse_rfc_r"] >=20]["reco_dataset_raw_data_sequence_te"].std(),
                                                 df.loc[df["reco_dataset_raw_data_sequence_pulse_rfc_r"] >=20]["reco_dataset_raw_data_sequence_te"].min(),
                                                 df.loc[df["reco_dataset_raw_data_sequence_pulse_rfc_r"] >=20]["reco_dataset_raw_data_sequence_te"].max()))

print("* For protocol(ii)?")
print("* TE (ms) = %.2f (%.2f) [%.2f - %.2f]" % (df.loc[df["reco_dataset_raw_data_sequence_pulse_rfc_r"] <20]["reco_dataset_raw_data_sequence_te"].mean(),
                                                df.loc[df["reco_dataset_raw_data_sequence_pulse_rfc_r"] <20]["reco_dataset_raw_data_sequence_te"].std(),
                                                df.loc[df["reco_dataset_raw_data_sequence_pulse_rfc_r"] <20]["reco_dataset_raw_data_sequence_te"].min(),
                                                df.loc[df["reco_dataset_raw_data_sequence_pulse_rfc_r"] <20]["reco_dataset_raw_data_sequence_te"].max()))


## Min and average TR

In [ ]:
print("n = %d" % len(df))

print("* Min TR (ms) = %.2f" % df["reco_dataset_raw_data_sequence_tr"].mean())

print("* For protocol(i)?")
print("* Min TR (ms) = %.2f" % df.loc[df["reco_dataset_raw_data_sequence_pulse_rfc_r"] >=20]["reco_dataset_raw_data_sequence_tr"].mean())

print("* For protocol(ii)?")
print("* Min TR (ms) = %.2f" % df.loc[df["reco_dataset_raw_data_sequence_pulse_rfc_r"] <20]["reco_dataset_raw_data_sequence_tr"].mean())


print("* Eff TR (s) = %.2f" % (df["acqtime_eff"] / df["reco_dataset_raw_data_sequence_na"]).mean())


## Acquisition time

In [ ]:
print("n = %d" % len(df))

print("* Acquisition time (min) = %.2f" % (df["acqtime"].mean() / 60))
print("* Eff acquisition time (min) = %.2f" % (df["acqtime_eff"].mean() / 60))

## Data discardment
### Average rejection rate

In [ ]:
df = df_sc_all_reco.loc[(df_sc_all_reco.index.get_level_values("reco_template_name") == "sc")].drop_duplicates(["patient_pass_id"])

print("n = %d" % len(df))
print("* Data rejected (%%) = %.2f" % df["rejection_rate_prct"].mean())

### Rejection method used the most

In [ ]:
plt.figure()
df["rejection_method"] = df["rejection_method"].astype(str)
df.groupby("rejection_method")["patient_pass_id"].nunique().plot.pie(autopct="%.1f%%")

### Gain in SNR/LW
#### Compare sc_nodatarej vs. sc_nodatarej_norea

In [ ]:
df = df_sc_all_reco.loc[df_sc_all_reco.index.get_level_values("reco_template_name").isin(["sc_nodatarej_norea", "sc_nodatarej"])]
df = df.reset_index().drop_duplicates(["patient_pass_id", "reco_template_name"])

df = df.reset_index().sort_values(by=["data_hash", "reco_template_name"], ascending=False)

df["snr_rel_diff"] = df.groupby(["data_hash"])["snr_final"].pct_change().fillna(0) * 100.0
df["lw_diff"] = df.groupby(["data_hash"])["lw_final"].diff().fillna(0)

print("--- sc_nodatarej vs. sc_nodatarej_norea ---")
print("n = %d" % len(df))
print("* SNR gain (%%) = %.2f" % (df.loc[df["reco_template_name"] == "sc_nodatarej"]["snr_rel_diff"].mean()))
print("* FWHM gain (Hz) = %.2f" % (df.loc[df["reco_template_name"] == "sc_nodatarej"]["lw_diff"].mean()))

df[["data_hash", "reco_template_name", "patient_pass_id", "snr_final", "lw_final", "snr_rel_diff", "lw_diff"]]

#### Compare sc vs. sc_nodatarej_norea

In [ ]:
df = df_sc_all_reco.loc[df_sc_all_reco.index.get_level_values("reco_template_name").isin(["sc_nodatarej_norea", "sc"])]
df = df.reset_index().drop_duplicates(["patient_pass_id", "reco_template_name"])

df = df.reset_index().sort_values(by=["data_hash", "reco_template_name"], ascending=False)

df["snr_rel_diff"] = df.groupby(["data_hash"])["snr_final"].pct_change().fillna(0) * 100.0
df["lw_diff"] = df.groupby(["data_hash"])["lw_final"].diff().fillna(0)

print("--- sc vs. sc_nodatarej_norea ---")
print("n = %d" % len(df))
print("* SNR gain (%%) = %.2f" % (df.loc[df["reco_template_name"] == "sc"]["snr_rel_diff"].mean()))
print("* FWHM gain (Hz) = %.2f" % (df.loc[df["reco_template_name"] == "sc"]["lw_diff"].mean()))

df[["data_hash", "reco_template_name", "patient_pass_id", "snr_final", "lw_final", "snr_rel_diff", "lw_diff"]]

* Frequency realignment is really benefic: +2%SNR but surtout -6Hz LW
* Data discardment performance is really not observable in terms of SNR/LW: just a little better LW but actually a loss in SNR...

# Investigating rejected datasets:

## [FIG] Normalized SNR vs. BMI

In [ ]:
df = df_sc_all_reco.loc[(df_sc_all_reco.index.get_level_values("reco_template_name") == "sc")].drop_duplicates("patient_pass_id")

this_df = df
this_df = this_df.rename(columns = {'snr_final_norm':'Normalized SNR (u.a)'})
this_df = this_df.rename(columns = {'patient_bmi':'Subject BMI'})
x = this_df["Subject BMI"]
y = this_df["Normalized SNR (u.a)"]

plt.figure()
g = sns.scatterplot(x="Subject BMI", y="Normalized SNR (u.a)",
                hue="voxel_pos_y_mm", cmap=cm,
                style="included", style_order=[True, False],
                s=150, data=this_df)

plt.grid(True)

norm = plt.Normalize(this_df['voxel_pos_y_mm'].min(), this_df['voxel_pos_y_mm'].max())
sm = plt.cm.ScalarMappable(cmap=cm, norm=norm)
sm.set_array([])

# Remove the legend and add a colorbar
g.get_legend().remove()
cb = g.figure.colorbar(sm)
cb.set_label(label='Y voxel position (mm)')

plt.show()

m, b = np.polyfit(x, y, 1)
plt.plot(x, m*x+b, color='k', linewidth=1)

print("R = %.2f" % x.corr(y))

plt.xlabel("Subject BMI (kg/m²)")

plt.savefig("./notebooks/figs/fig_snr_bmi.svg")

## 5ppm artefact?

In [ ]:
plt.figure()
g = sns.boxplot(x="included", y="artefact_signal_norm", data=df)
plt.grid('on')

Clearly, rejected datasets show a stronger 5ppm artefact level in average... It is one of the reasons yes. 

# [FIG] Spectra gallery

In [ ]:
this_df = df_sc_all_reco.loc[(df_sc_all_reco.index.get_level_values("reco_template_name") == "sc") &
                        (df_sc_all_reco["fit_tool_name"] == "pastis") &
                        (df_sc_all_reco["fit_ws_metabolites_len"] == 17) &
                        (df_sc_all_reco["fit_ws_sequence_str"] == "none")].drop_duplicates("patient_pass_id")

this_df = this_df.loc[this_df["patient_pass_id"].isin(["338_P1", "314_P2", "311_P1", "300_P2"])]

# add best brain dataset
this_df_brain = df_brain.loc[(df_brain.index.get_level_values("reco_template_name") == "brain") &
                        (df_brain["fit_tool_name"] == "pastis") &
                        (df_brain["fit_ws_metabolites_len"] == 17) &
                        (df_brain["fit_ws_sequence_str"] == "none")].drop_duplicates("patient_pass_id")

this_df_brain = this_df_brain.loc[ (this_df_brain["snr_final"] == this_df_brain["snr_final"].max())]

# concat
this_df = pd.concat([this_df, this_df_brain])
this_df = this_df.sort_values(by="snr_final")

# add SNR to the label
this_df["spectra_gllr_lbl"] = ["%s (%.1f)" % (s1.replace(" (P1)", "").replace(" (P2)", ""), s2) for (s1, s2) in zip(this_df["patient_pass_id_pretty"], this_df["snr_final"])]

k = 0
label_x = 1.9 #ppm
for this_index, this_row in this_df.iterrows():
    s = this_row["data_obj"]
    
    s.set_display_offset(10000.0 * k)
    fig = s.display_spectrum_1d(1000)
    
    # add some x/y for annotation
    this_df.at[this_index, "spectrum_label_x"] = label_x
    this_df.at[this_index, "spectrum_label_y"] = 10000.0 * k
    
    # line color
    if(this_row["included"]):
        fig.axes[0].get_lines()[-1].set_color('black')
    else:
        fig.axes[0].get_lines()[-1].set_color('grey')
        
    # iterate
    k += 1

add_labels("spectrum_label_x", "spectrum_label_y", "spectra_gllr_lbl", this_df, plt.gca(),  9, 0, 3500)

ax = fig.axes
ax[0].set(yticklabels=[])
ax[0].get_legend().remove()
plt.ylabel("")
  
plt.xlim([5, 1])
yts = plt.yticks()
plt.yticks(yts[0][0:-4])
plt.ylim([-5000, 75000])

# add metabolites labels
yo = 10000
plt.text(2, 60000 + yo, "tNAA")
plt.text(2.5, 35000 + yo, "Glx")
plt.text(2.85, 33000 + yo, "NAA")
plt.text(3, 50000 + yo, "tCr")
plt.text(3.55, 45000 + yo, "tCho")
plt.text(3.52, 34000 + yo, "Tau")
plt.text(4.15, 45000 + yo, "tCr")
plt.text(4.25, 35000 + yo, "mI")
plt.text(3.65, 38000 + yo, "mI")
plt.text(3.9, 38000 + yo, "Glu")


In [ ]:
# get previous fig and paste it here
new_fig = plt.figure(figsize=(13,6))
new_fig.subplots_adjust(wspace=0.01, hspace=0.01)

fig = plt.figure(1000)
ax_previous_fig = fig.axes[0]

ax_previous_fig.figure=new_fig
new_fig.axes.append(ax_previous_fig)
new_fig.add_axes(ax_previous_fig)

dummy = new_fig.add_subplot(1,3,(1,2))
ax_previous_fig.set_position(dummy.get_position())
dummy.remove()

xlim = ax_previous_fig.get_xlim()
ylim = ax_previous_fig.get_ylim()
ax_previous_fig.text(xlim[0] - (xlim[0]-xlim[1])*0.02, 
                     ylim[0] + (ylim[1]-ylim[0])*0.02, 
                    "A.",
                    fontsize=12)

# add a nice anatomy for spinal cord
t2w = suspect.image.load_dicom_volume("/home/tangir/crmbm/acq/314-yt-p2-moelle/20200625/01_0005_t2-tse-sag-2d-10sl-p2-trig-s4-nd/original-primary-m-norm-nd_e01_0001.dcm")

ind_patient = 2
this_df_nice_subject = this_df.loc[this_df["patient_pass_id"] == "314_P2"]
pcg = this_df_nice_subject["data_obj"].iloc[0]
print(this_df_nice_subject["patient_pass_id"].iloc[0])
print(this_df_nice_subject["patient_id_pretty"].iloc[0])

pcg_centre = pcg.to_scanner(0, 0, 0)
pcg_centre_index = t2w.from_scanner(*pcg_centre).round().astype(int)

corner_coords_pcg = [[0, -0.4, -1],
                     [0, -0.4, 1],
                     [0, 0.4, 1],
                     [0, 0.4, -1],
                     [0, -0.4, -1]]
corner_coords = np.array([t2w.from_scanner(*pcg.to_scanner(*coord)) for coord in corner_coords_pcg])


ax = new_fig.add_subplot(2,2,4)
ax.imshow(t2w[pcg_centre_index[2]], cmap=plt.cm.gray)
ax.plot(corner_coords[:, 0], corner_coords[:, 1], 'red')
ax.set_xticks([])
ax.set_yticks([])
new_fig.subplots_adjust(wspace=0.1, hspace=0.01)

xlim = ax.get_xlim()
ylim = ax.get_ylim()
ax.text(xlim[0] - (xlim[0]-xlim[1])*0.03, 
        ylim[0] + (ylim[1]-ylim[0])*0.06, 
        "C.",
        fontsize=12, color="white")

In [ ]:
# add a nice anatomy for brain

t1w = suspect.image.load_dicom_volume("/home/tangir/crmbm/acq/347-re-p1-moelle/20200123/01_0009_mp2rage-wip900-sag-0-6iso-p3-uni-den/original-primary-m-nd-uni_e01_0001.dcm")

pcg = reco.MRSData2("/home/tangir/crmbm/acq_twix/347-re-p1-moelle/meas_MID228_slaser_R_N=10_2_longTE_SNR+++_FID50587.dat")

pcg_centre = pcg.to_scanner(0, 0, 0)
pcg_centre_index_brain = t1w.from_scanner(*pcg_centre).round().astype(int)

corner_coords_pcg = [[0, -0.75, -0.75],
                     [0, -0.75,  0.75],
                     [0, 0.75, 0.75],
                     [0, 0.75, -0.75],
                     [0, -0.75, -0.75]]
corner_coords = np.array([t1w.from_scanner(*pcg.to_scanner(*coord)) for coord in corner_coords_pcg])

ax = new_fig.add_subplot(2,2,2)
img_tmp = t1w[:, pcg_centre_index_brain[1], :].T
img_tmp = np.concatenate((np.zeros([402, int((402-256)/2)]), img_tmp), axis=1)
img_tmp = np.concatenate((img_tmp, np.zeros([402, int((402-256)/2)])), axis=1)

ax.imshow(img_tmp, cmap=plt.cm.gray)
ax.plot(corner_coords[:, 1]+int((402-256)/2), corner_coords[:, 0], 'red')
ax.set_xticks([])
ax.set_yticks([])
new_fig.subplots_adjust(wspace=0.1, hspace=0.01)

xlim = ax.get_xlim()
ylim = ax.get_ylim()
ax.text(xlim[0] - (xlim[0]-xlim[1])*0.03, 
        ylim[0] + (ylim[1]-ylim[0])*0.06, 
        "B.",
        fontsize=12, color="white")


new_fig.savefig("./notebooks/figs/fig_spectra.svg")

# [FIG] Fit example
with the best fit we have :)

In [ ]:
import mrs.fit as fit

this_df = df_sc_all_reco.loc[(df_sc_all_reco.index.get_level_values("reco_template_name") == "sc") &
                        (df_sc_all_reco["fit_tool_name"] == "pastis") &
                        (df_sc_all_reco["fit_ws_metabolites_len"] == 17) &
                        (df_sc_all_reco["fit_ws_sequence_str"] == "none")].drop_duplicates("patient_pass_id")

# preview
this_df = this_df.sort_values("fit_ws_optim_results_rsq_f", ascending=False).drop_duplicates("patient_pass_id")
this_df = this_df.head(5)

# best
this_df = this_df.loc[this_df["patient_pass_id"].isin(["314_P2"])]

for this_index, this_row in this_df.iterrows():
    s_data = this_row["data_obj"]
    s_data = s_data.correct_peak_removal_1d(8, [4.5, 6], False)

    # fit params
    p_fit = this_row["final_fit_params_arr"]

    # PRESS seq
    seq = sim.mrs_seq_press(s_data.sequence.te, s_data.sequence.tr, s_data.sequence.na, 
                            s_data.sequence.ds, s_data.sequence.nuclei, 
                            s_data.sequence.npts, s_data.sequence.voxel_size, 
                            s_data.sequence.fs, s_data.sequence.f0)
    
    seq.bandpass_filter_range_ppm = [0, 4.2]

    seq.initialize()
    fig = plt.figure()
    ax = fig.subplots()
    fit.disp_fit(ax, s_data, p_fit, seq, True, True, None, False, [1, 4.5])
    print(this_row["patient_pass_id"] + " R(t/f) = %.2f/%.2f FQN = %.1f" % (this_row["fit_ws_optim_results_rsq_t"],
                                                                            this_row["fit_ws_optim_results_rsq_f"],
                                                                            this_row["fit_ws_optim_results_fqn"]))

fig = plt.gcf()
fig.savefig("./notebooks/figs/fig_fit.svg")

# P1 quantification results
## Ratio STD: choosing best modelization sequence

In [ ]:
df = df_sc_all_reco.loc[(df_sc_all_reco["included"]) & 
                        (df_sc_all_reco.index.get_level_values("reco_template_name") == "sc") & 
                        (df_sc_all_reco["pass"] == 1)]

# watch ratios
this_df = df.loc[(df["param_p_type"] == "fit_ws_params_fit_t2cor_ratio_cm") &
                 (df["param_val_err"] == "val") &  
                 (df["param_m_name"].isin(["tCr", "tCho", "tNAA", "mI", "Tau", "Gln", "Glu", "GABA"]))]

this_df["param_val"] = this_df["param_val"].replace([np.inf, -np.inf], np.nan)
this_df = this_df[this_df["param_val"].notna()]
this_df = this_df.groupby(["param_m_name", "strategy"])["param_val"].std()

plt.figure()
g = sns.barplot(x="param_m_name", 
                y="param_val",
                hue="strategy",
                palette = "bright",
                data=this_df.reset_index())


Clearly, the use of PRESS instead of sLASER for the modelization increases the variability of the results. Let's stick to sLASER ("none").

## Ratio STD & CRB: choosing number of metabolites

In [ ]:
df = df_sc_all_reco.loc[(df_sc_all_reco["included"]) & 
                        (df_sc_all_reco.index.get_level_values("reco_template_name") == "sc") &
                        (df_sc_all_reco["fit_ws_sequence_str"] == "none") & 
                        (df_sc_all_reco["pass"] == 1)]

# watch ratios
this_df = df.loc[(df["param_p_type"] == "fit_ws_params_fit_t2cor_ratio_cm") &
                 (df["param_val_err"] == "val") &  
                 (df["param_m_name"].isin(["tCr", "tCho", "tNAA", "mI", "Tau", "Gln", "Glu", "GABA", "NAAG"]))]

this_df["param_val"] = this_df["param_val"].replace([np.inf, -np.inf], np.nan)
this_df = this_df[this_df["param_val"].notna()]
this_df = this_df.groupby(["param_m_name", "strategy"])["param_val"].std()

plt.figure()
g = sns.barplot(x="param_m_name", 
                y="param_val",
                hue="strategy",
                palette = "bright",
                data=this_df.reset_index())


In [ ]:
df = df_sc_all_reco.loc[(df_sc_all_reco["included"]) & 
                        (df_sc_all_reco.index.get_level_values("reco_template_name") == "sc") &
                        (df_sc_all_reco["fit_ws_sequence_str"] == "none") & 
                        (df_sc_all_reco["pass"] == 1)]

# watch ratios
this_df = df.loc[(df["param_p_type"] == "fit_ws_params_fit_t2cor_ratio_cm") &
                 (df["param_val_err"] == "err_prct") &  
                 (df["param_m_name"].isin(["tCr", "tCho", "tNAA", "mI", "Tau", "Gln", "Glu", "GABA", "NAAG"]))]

this_df["param_val"] = this_df["param_val"].replace([np.inf, -np.inf], np.nan)
this_df = this_df[this_df["param_val"].notna()]
this_df = this_df.groupby(["param_m_name", "strategy"])["param_val"].mean()

plt.figure()
g = sns.barplot(x="param_m_name", 
                y="param_val",
                hue="strategy",
                palette = "bright",
                data=this_df.reset_index())

plt.ylim([0, 100])

15 metabolites gives the lowest STDs and CRBs !

## Automatic adjustment of SNR/LW/FQN thresholds

In [ ]:
threshold_snr_final_list = [5, 6, 7, 8, 9, 10]
threshold_lw_final_list = [15, 20, 25]
threshold_fqn_final_list = [1.25, 1.5, 2]

ref_reco_template_name_sc = "sc"
ref_fit_tool_name = "pastis"
ref_fit_ws_metabolites_len = 17
ref_fit_ws_sequence_str = "none"

this_df_reslist = []
k = 0
for this_snr in threshold_snr_final_list:
    for this_lw in threshold_lw_final_list:
        for this_fqn in threshold_fqn_final_list:

            df_sc_all_reco = filter_by_SNR_LW_FQN(df_sc_all_reco, 
                                                    this_snr,
                                                    this_lw,
                                                    this_fqn,
                                                    ref_reco_template_name_sc, 
                                                    ref_fit_tool_name, 
                                                    ref_fit_ws_metabolites_len, 
                                                    ref_fit_ws_sequence_str)

            df = df_sc_all_reco.loc[(df_sc_all_reco["included"]) & 
                                    (df_sc_all_reco.index.get_level_values("reco_template_name") == "sc") &
                                    (df_sc_all_reco["fit_ws_sequence_str"] == "none") &
                                    (df_sc_all_reco["fit_ws_metabolites_len"] == 17) &
                                    (df_sc_all_reco["fit_tool_name"] == "pastis") &
                                    (df_sc_all_reco["pass"] == 1)]

            # watch ratios
            this_df = df.loc[(df["param_p_type"] == "fit_ws_params_fit_t2cor_ratio_cm") &
                             (df["param_val_err"] == "val") &  
                             (df["param_m_name"].isin(["tCr", "tCho", "tNAA", "mI", "Tau", "Gln", "Glu", "GABA", "NAAG"]))]

            this_df["param_val"] = this_df["param_val"].replace([np.inf, -np.inf], np.nan)
            this_df = this_df[this_df["param_val"].notna()].reset_index()

            this_df = this_df.loc[(this_df["param_m_name"].isin(["tCho", "tNAA", "mI"]))]
            this_df_res = this_df.groupby(["param_m_name"])["param_val"].agg(["mean", "median", "std"])
            this_df_res["snr_threshold"] = this_snr
            this_df_res["lw_threshold"] = this_lw
            this_df_res["fqn_threshold"] = this_fqn
            this_df_reslist.append(this_df_res)

            k += 1
            print("%.2f%% done" % (100 * k / (len(threshold_snr_final_list) * len(threshold_lw_final_list) * len(threshold_fqn_final_list))))


In [ ]:
this_df_res_concat = pd.concat(this_df_reslist).reset_index()
this_df_res_concat = this_df_res_concat.loc[(this_df_res_concat["lw_threshold"] == this_df_res_concat["lw_threshold"].max())]

plt.figure()
g = sns.lineplot(x="snr_threshold", y="std",
                hue="param_m_name",
                style="fqn_threshold", markers=True,
                data=this_df_res_concat)

this_df_res_concat = pd.concat(this_df_reslist).reset_index()
this_df_res_concat = this_df_res_concat.loc[(this_df_res_concat["fqn_threshold"] == this_df_res_concat["fqn_threshold"].max())]

plt.figure()
g = sns.lineplot(x="snr_threshold", y="std",
                hue="param_m_name",
                style="lw_threshold", markers=True,
                data=this_df_res_concat)

this_df_res_concat = pd.concat(this_df_reslist).reset_index()
this_df_res_concat = this_df_res_concat.loc[(this_df_res_concat["snr_threshold"] == 8)]

plt.figure()
g = sns.lineplot(x="fqn_threshold", y="std",
                hue="param_m_name",
                style="lw_threshold", markers=True,
                data=this_df_res_concat)


## Final stats: Mean, Median, STDs, CRBs, pastis vs. LCModel, sc vs. sc_nodatarej

In [ ]:
threshold_snr_final = 5
threshold_lw_final = 20
threshold_fqn_final = 200

ref_reco_template_name_sc = "sc"
ref_fit_tool_name = "pastis"
ref_fit_ws_metabolites_len = 17
ref_fit_ws_sequence_str = "none"

df_sc_all_reco = filter_by_SNR_LW_FQN(df_sc_all_reco, 
                                        threshold_snr_final,
                                        threshold_lw_final,
                                        threshold_fqn_final,
                                        ref_reco_template_name_sc, 
                                        ref_fit_tool_name, 
                                        ref_fit_ws_metabolites_len, 
                                        ref_fit_ws_sequence_str)

df = df_sc_all_reco.loc[(df_sc_all_reco["included"]) & 
                        df_sc_all_reco.index.get_level_values("reco_template_name").isin(["sc", "sc_nodatarej"]) &
                        (df_sc_all_reco["fit_ws_sequence_str"] == "none") &
                        df_sc_all_reco["fit_ws_metabolites_len"].isin([1,15]) & 
                        (df_sc_all_reco["pass_after_filter"] == 1)]


In [ ]:
# watch SDs
this_df = df.loc[(df["param_p_type"] == "fit_ws_params_fit_t2cor_ratio_cm") &
                 (df["param_val_err"] == "val") &  
                 (df["param_m_name"].isin(["tCho", "tNAA", "mI", "Tau", "Gln", "Glu"]))]

this_df["param_val"] = this_df["param_val"].replace([np.inf, -np.inf], np.nan)
this_df = this_df[this_df["param_val"].notna()].reset_index()

this_df_grp_disp = this_df.groupby(["param_m_name", "strategy"])["param_val"].std()

this_df.groupby(["param_m_name", "strategy"])["param_val"].agg(["mean", "std"])

In [ ]:
plt.figure()
g = sns.barplot(x="param_m_name", 
                y="param_val",
                hue="strategy",
                palette = "bright",
                data=this_df_grp_disp.reset_index())

In [ ]:
# watch CRBs
this_df = df.loc[(df["param_p_type"] == "fit_ws_params_fit_t2cor_ratio_cm") &
                 (df["param_val_err"] == "err_prct") &  
                 (df["param_m_name"].isin(["tCho", "tNAA", "mI", "Tau", "Gln", "Glu"]))]

this_df["param_val"] = this_df["param_val"].replace([np.inf, -np.inf], np.nan)
this_df = this_df[this_df["param_val"].notna()].reset_index()

this_df_grp = this_df.groupby(["param_m_name", "strategy"])["param_val"].median()

this_df_grp

In [ ]:
plt.figure()
g = sns.barplot(x="param_m_name", 
                y="param_val",
                hue="strategy",
                palette = "bright",
                data=this_df_grp.reset_index())

plt.ylim([0, 100])

# P1 vs. P2 quantification results

In [ ]:
threshold_snr_final = 5
threshold_lw_final = 20
threshold_fqn_final = 200

ref_reco_template_name_sc = "sc"
ref_fit_tool_name = "pastis"
ref_fit_ws_metabolites_len = 17
ref_fit_ws_sequence_str = "none"

df_sc_all_reco = filter_by_SNR_LW_FQN(df_sc_all_reco, 
                                        threshold_snr_final,
                                        threshold_lw_final,
                                        threshold_fqn_final,
                                        ref_reco_template_name_sc, 
                                        ref_fit_tool_name, 
                                        ref_fit_ws_metabolites_len, 
                                        ref_fit_ws_sequence_str)

df = df_sc_all_reco.loc[(df_sc_all_reco["included"]) & 
                        df_sc_all_reco.index.get_level_values("reco_template_name").isin(["sc", "sc_nodatarej"]) &
                        (df_sc_all_reco["fit_ws_sequence_str"] == "none") &
                        df_sc_all_reco["fit_ws_metabolites_len"].isin([1,15])]

# watch ratios
this_df = df.loc[(df["param_p_type"] == "fit_ws_params_fit_t2cor_ratio_cm") &
                 (df["param_val_err"] == "val") &  
                 (df["param_m_name"].isin(["tCho", "tNAA", "mI", "Tau", "Gln", "Glu"]))]

# keep only patients that has P1 and P2
# find pass 1 patients
this_df_p1 = this_df.loc[(this_df["pass_after_filter"] == 1)]
# remember them
patient_id_p1_list = this_df_p1["patient_id"].unique().tolist()
# find pass 2 patients which are in previously found pass 1 list
patient_id_p1p2_list = this_df.loc[ (this_df["pass_after_filter"] == 2) &
                            this_df["patient_id"].isin(patient_id_p1_list) ]["patient_id"].unique().tolist()
# select those p1p2 patients
this_df_p1p2 = this_df.loc[ this_df["patient_id"].isin(patient_id_p1p2_list) ]

this_df_p1p2["param_val"] = this_df_p1p2["param_val"].replace([np.inf, -np.inf], np.nan)
this_df_p1p2 = this_df_p1p2[this_df_p1p2["param_val"].notna()]
this_df_p1p2[["patient_id", "pass_after_filter", 'param_val']]
this_df_p1p2_grp = this_df_p1p2.groupby(["param_m_name", "strategy", "patient_id"])["param_val"].std() / this_df_p1p2.groupby(["param_m_name", "strategy", "patient_id"])["param_val"].mean() * 100.0
this_df_p1p2_grp.groupby(["param_m_name", "strategy"]).mean()

In [ ]:
plt.figure()
g = sns.barplot(x="param_m_name", 
                y="param_val",
                hue="strategy",
                palette = "bright",
                data=this_df_p1p2_grp.reset_index())

plt.ylim([0, 100])